**0ption 05 - Fraud Detection for Finance**




*   step: maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).
*   customer: ID of the customer
*   age: The age of the accounts
*   gender: Gender of the customer
*   zipcodeOri: The zip code of the customer
*   merchant: ID of the merchant
*   zipMerchant: The zip code of the merchant
*   category: Category of the transaction
*   amount: The amount of the transaction
*   fraud: Binary value indicating fraudulent or not fraudulent


In [200]:
# (1) Import libraries
import pandas as pd
from sklearn.model_selection import  train_test_split
from pandas import DataFrame
import numpy as np

# Import the class of the machine learning model
from sklearn.ensemble import RandomForestClassifier

# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

# Import metrics for measuring performance of the model
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [201]:
!curl -O https://storage.googleapis.com/datascience-practice/bs140513_032310.csv.zip
!curl -O https://storage.googleapis.com/datascience-practice/bsNET140513_032310.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7251k  100 7251k    0     0  18.9M      0 --:--:-- --:--:-- --:--:-- 18.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6139k  100 6139k    0     0  17.7M      0 --:--:-- --:--:-- --:--:-- 17.7M


In [202]:
# (2.a) See the downloaded files
!ls

bs140513_032310.csv	 bsNET140513_032310.csv.zip	  sample_data
bs140513_032310.csv.zip  predicted.csv
bsNET140513_032310.csv	 random_forest_classifier.joblib


In [203]:
# (2.b) Unzip the files
!unzip bs140513_032310.csv.zip
!unzip bsNET140513_032310.csv.zip

Archive:  bs140513_032310.csv.zip
replace bs140513_032310.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  bsNET140513_032310.csv.zip
replace bsNET140513_032310.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [204]:
# (2.c) Read transaction data into Pandas DataFrame
dft = pd.read_csv('bs140513_032310.csv')
# (2.d) Read network data into Pandas DataFrame
dff = pd.read_csv('bsNET140513_032310.csv')

In [205]:
dft.head(10)


,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0
5,0,'C1315400589','3','F','28007','M348934600','28007','es_transportation',25.81,0
6,0,'C765155274','1','F','28007','M348934600','28007','es_transportation',9.10,0
7,0,'C202531238','4','F','28007','M348934600','28007','es_transportation',21.17,0
8,0,'C105845174','3','M','28007','M348934600','28007','es_transportation',32.40,0
9,0,'C39858251','5','F','28007','M348934600','28007','es_transportation',35.40,0


In [206]:
dff.head(10)

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0
5,'C1315400589','M348934600',25.81,'es_transportation',0
6,'C765155274','M348934600',9.10,'es_transportation',0
7,'C202531238','M348934600',21.17,'es_transportation',0
8,'C105845174','M348934600',32.40,'es_transportation',0
9,'C39858251','M348934600',35.40,'es_transportation',0


In [207]:
# (3) Explore data (for example, see what is categorial and numerical)
dft.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB


In [208]:
# Finding total of uniques values for all categorical features
categoricalFeatures = ['customer','age','gender','zipcodeOri','merchant','zipMerchant','category']
for value in categoricalFeatures:
  print(value + ':',len(dft[value].unique()))

customer: 4112
age: 8
gender: 4
zipcodeOri: 1
merchant: 50
zipMerchant: 1
category: 15


In [209]:
print(np.unique(dft[['gender']].values))

["'E'" "'F'" "'M'" "'U'"]


In [210]:
irrelevantFeatures = ['zipcodeOri','zipMerchant']
dft_relevant=dft.drop(irrelevantFeatures, axis=1, inplace=False)

In [211]:
dft_relevant.head(10)

,step,customer,age,gender,merchant,category,amount,fraud
0,0,'C1093826151','4','M','M348934600','es_transportation',4.55,0
1,0,'C352968107','2','M','M348934600','es_transportation',39.68,0
2,0,'C2054744914','4','F','M1823072687','es_transportation',26.89,0
3,0,'C1760612790','3','M','M348934600','es_transportation',17.25,0
4,0,'C757503768','5','M','M348934600','es_transportation',35.72,0
5,0,'C1315400589','3','F','M348934600','es_transportation',25.81,0
6,0,'C765155274','1','F','M348934600','es_transportation',9.10,0
7,0,'C202531238','4','F','M348934600','es_transportation',21.17,0
8,0,'C105845174','3','M','M348934600','es_transportation',32.40,0
9,0,'C39858251','5','F','M348934600','es_transportation',35.40,0


In [212]:
# From Category to numeric
if(dft_relevant['age'].dtype == 'object'):
  dft_relevant['age']= dft_relevant['age'].astype('category')
  dft_relevant['age'] = dft_relevant['age'].cat.codes

if(dft_relevant['gender'].dtype == 'object'):
  dft_relevant['gender']= dft_relevant['gender'].astype('category')
  dft_relevant['gender'] = dft_relevant['gender'].cat.codes


In [213]:
# One-Hot Encoding
dft_encoded = pd.get_dummies(dft_relevant, columns=["merchant","category"])

# The "customer" variable wasn't included in One-Hot Encoding, because it consumes 2.3 gb and requires Google Colab Pro version


In [214]:
dft_encoded.head()

,step,customer,age,gender,amount,fraud,merchant_'M1053599405',merchant_'M117188757',merchant_'M1198415165',merchant_'M1294758098',...,category_'es_home',category_'es_hotelservices',category_'es_hyper',category_'es_leisure',category_'es_otherservices',category_'es_sportsandtoys',category_'es_tech',category_'es_transportation',category_'es_travel',category_'es_wellnessandbeauty'
0,0,'C1093826151',4,2,4.55,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,'C352968107',2,2,39.68,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,'C2054744914',4,1,26.89,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,'C1760612790',3,2,17.25,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,'C757503768',5,2,35.72,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [215]:
dft_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 71 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   step                              594643 non-null  int64  
 1   customer                          594643 non-null  object 
 2   age                               594643 non-null  int8   
 3   gender                            594643 non-null  int8   
 4   amount                            594643 non-null  float64
 5   fraud                             594643 non-null  int64  
 6   merchant_'M1053599405'            594643 non-null  uint8  
 7   merchant_'M117188757'             594643 non-null  uint8  
 8   merchant_'M1198415165'            594643 non-null  uint8  
 9   merchant_'M1294758098'            594643 non-null  uint8  
 10  merchant_'M1313686961'            594643 non-null  uint8  
 11  merchant_'M1352454843'            594643 non-null  u

In [216]:
dft_encoded.describe()

,step,age,gender,amount,fraud,merchant_'M1053599405',merchant_'M117188757',merchant_'M1198415165',merchant_'M1294758098',merchant_'M1313686961',...,category_'es_home',category_'es_hotelservices',category_'es_hyper',category_'es_leisure',category_'es_otherservices',category_'es_sportsandtoys',category_'es_tech',category_'es_transportation',category_'es_travel',category_'es_wellnessandbeauty'
count,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,...,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000,594643.000000
mean,94.986827,3.014151,1.451089,37.890135,0.012108,0.011471,0.000035,0.002657,0.000321,0.000886,...,0.003340,0.002933,0.010255,0.000839,0.001534,0.006730,0.003986,0.849449,0.001224,0.025370
std,51.053632,1.335884,0.503291,111.402831,0.109369,0.106486,0.005943,0.051478,0.017919,0.029757,...,0.057695,0.054076,0.100746,0.028956,0.039132,0.081761,0.063006,0.357611,0.034968,0.157246
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,52.000000,2.000000,1.000000,13.740000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,97.000000,3.000000,1.000000,26.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,139.000000,4.000000,2.000000,42.540000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,179.000000,7.000000,3.000000,8329.960000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [217]:
#  Choose the label and features
labelDft = dft_encoded['fraud']
NoFeatures=['customer','fraud']
featureDft = dft_encoded.drop(NoFeatures,axis=1,inplace=False)


In [218]:
# Training, Testing (and/or Validation) data split 
trainF, testF, trainL, testL =train_test_split(featureDft,labelDft, test_size=0.2,random_state=0)

In [219]:
# Training the Machine Learning Model (i.e, Fitting the Model)

# Create instance of RandomForestClassifier model
rf_model = RandomForestClassifier()

# Fitting the Model
rf_model.fit( trainF, trainL.values.ravel() )

RandomForestClassifier()

In [220]:
# Predict Train Data
predLabelTrain = rf_model.predict(trainF)


In [221]:
# Evaluate the model metrics for Training (and/or Validation) data
accuracy = round(accuracy_score(trainL, predLabelTrain), 3)
precision = round(precision_score(trainL, predLabelTrain), 3)
recall = round(recall_score(trainL, predLabelTrain), 3)

print("accuracy(train):",accuracy)
print('precision(train):',precision)
print('recall(train):',recall)

accuracy(train): 1.0
precision(train): 1.0
recall(train): 1.0


In [222]:
# Predict Test Data
predLabelTest = rf_model.predict(testF)

In [223]:
# Evaluate the model metrics for Test data
accuracy = round(accuracy_score(testL, predLabelTest), 3)
precision = round(precision_score(testL, predLabelTest), 3)
recall = round(recall_score(testL, predLabelTest), 3)
print("accuracy(test):",accuracy)
print('precision(test):',precision)
print('recall(test):',recall)

accuracy(test): 0.996
precision(test): 0.863
recall(test): 0.78


**Saving the Model**

In [169]:
%pip install pickle
%pip install joblib

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [224]:
# Converts to binary
import pickle

In [225]:
# Saving and loading the model parameters
from joblib import dump, load
model_parameters = dump(rf_model, './random_forest_classifier.joblib')

**Loading the Model**

In [226]:
rf_model_loaded = load('./random_forest_classifier.joblib')

In [227]:
# Use the model for prediction
predictions = rf_model_loaded.predict(trainF)

print(predictions)

[0 0 0 ... 0 0 0]


In [228]:
predictions_df = pd.DataFrame(data=predictions)
predictions_df


,0
0,0
1,0
2,0
3,0
4,0
...,...
475709,0
475710,0
475711,0
475712,0


In [229]:
# Write final predicted data  (e.g, to CSV or JSON, etc.)
trainF['prediction']= predictions_df
trainF.to_csv('./predicted.csv',index=False)

In [230]:
trainF.head()

,step,age,gender,amount,merchant_'M1053599405',merchant_'M117188757',merchant_'M1198415165',merchant_'M1294758098',merchant_'M1313686961',merchant_'M1352454843',...,category_'es_hotelservices',category_'es_hyper',category_'es_leisure',category_'es_otherservices',category_'es_sportsandtoys',category_'es_tech',category_'es_transportation',category_'es_travel',category_'es_wellnessandbeauty',prediction
74368,27,2,1,49.42,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0
332888,108,2,2,50.04,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0
396963,126,4,1,5.21,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0
251778,84,1,1,20.39,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0
221582,75,4,1,14.36,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0
